# 구글플레이에 등록된 앱의 리뷰 추출

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

# 웹브라우저를 실행 할 때 사용할 옵션 - 사람인 것처럼 정보 입력
options = Options()
options.add_experimental_option("detach", True)
options.add_argument("start-maximized")
options.add_argument("Chrome/141.0.0.0")
options.add_argument("lang=ko_KR")


driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
    )
# 웹브라우저에서 url 열기
driver.get("https://play.google.com/store/apps/details?id=viva.republica.toss")

In [2]:
import time

In [3]:
driver.execute_script(f"window.scrollTo({0}, {1400})")
time.sleep(2)

In [4]:
# 평점 및 리뷰 옆의 -> 버튼 클릭
button = driver.find_element(By.CSS_SELECTOR, 'button[aria-label*="평점 및 리뷰 자세히 알아보기"]')
button.click()


In [5]:
# 최신순으로 리뷰를 정렬하기 위해서 버튼 클릭
driver.find_element(By.ID, "sortBy_1").click()


In [6]:
# 최신을 찾아 클릭
driver.find_element(By.CSS_SELECTOR, 'span[aria-label*="최신"]').click()

In [7]:
# window를 스크롤하면 리뷰가 아닌 배경 페이지가 스크롤됨
# driver.execute_script(f"window.scrollTo({0}, {2400})")
# time.sleep(2)

In [8]:
# 리뷰가 담긴 창을 찾아서 Javascript로 1000px씩 아래로 스크롤
driver.execute_script("document.querySelector('.fysCi.Vk3ZVd').scrollBy(0, 10000)")

In [9]:
# 리뷰일
review_date = driver.find_element(By.CSS_SELECTOR, ".bp9Aid").get_attribute("innerHTML")

In [10]:
# 별점
rating = float(driver.find_element(By.CSS_SELECTOR, 'div[aria-label*="별표 5개 만점에"]').get_attribute("aria-label").split()[3].replace("개를", ""))

In [11]:
# 사용자 리뷰
user_review = driver.find_element(By.CSS_SELECTOR, ".h3YV2d").get_attribute("innerHTML")

In [12]:
# 회사 응답
company_reply = driver.find_element(By.CSS_SELECTOR, ".ras4vb > div").get_attribute("innerHTML")

In [13]:
from datetime import datetime, timedelta

In [14]:
# 추출한 리뷰일을 날짜형 데이터로 변경
# datetime.srtptime(yyyy-mm-dd, "%Y-%m-%d") 날짜형 데이터 타입으로 변환
review_date = review_date.replace(" ", "").replace("년", "-").replace("월", "-").replace("일", "")
review_date = datetime.strptime(review_date, "%Y-%m-%d")

In [15]:
review_date.date()

datetime.date(2025, 6, 14)

In [16]:
today = datetime.today()
one_month_ago = today - timedelta(days=30)
one_month_ago.date()

datetime.date(2026, 1, 12)

# 코드 합쳐서 토스 리뷰 1달치 수집하기

In [18]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

웹브라우저에 표시되는 요소들(버튼, 아이콘)은 클릭할 수 있게 되기까지 시간이 필요.<br>
요소가 클릭이 가능해질 때 까지 기다리도록 코드를 작성해 주어야 함<br>
```python
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
```
추가가 필요

In [19]:
import time
import pandas as pd
from datetime import datetime, timedelta
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [20]:
def to_date(review_date):
    # datetime.srtptime(yyyy-mm-dd, "%Y-%m-%d") 날짜형 데이터 타입으로 변환
    review_date = review_date.replace(" ", "").replace("년", "-").replace("월", "-").replace("일", "")
    review_date = datetime.strptime(review_date, "%Y-%m-%d")
    return review_date.date()

In [21]:
apps = dict(토스="viva.republica.toss", KB스타뱅킹="com.kbstar.kbbank", 하나원큐="com.kebhana.hanapush", 뱅크샐러드="com.rainist.banksalad2", 핀다="kr.co.finda.finda")

In [22]:
def app_review_extractor(app, days=7):
    # 웹브라우저를 실행 할 때 사용할 옵션 - 사람인 것처럼 정보 입력
    options = Options()

    # ✅ 브라우저가 꺼지지 않도록 유지
    options.add_experimental_option("detach", True)

    # ✅ 실제 사용자 환경과 유사한 기본 설정
    options.add_argument("--start-maximized")                # 최대화된 창으로 시작
    options.add_argument("--disable-blink-features=AutomationControlled")  # 자동화 탐지 회피
    options.add_argument("--disable-infobars")               # “Chrome이 자동 테스트 중” 문구 제거
    options.add_argument("--disable-extensions")             # 불필요한 확장 기능 제거
    options.add_argument("--disable-popup-blocking")         # 팝업 차단 비활성화 (사람처럼 행동)
    options.add_argument("--no-sandbox")                     # 보안 샌드박스 비활성화
    options.add_argument("--disable-dev-shm-usage")          # 리눅스 환경 안정성 향상

    # ✅ 실제 브라우저 User-Agent (사람의 브라우저처럼 보이도록)
    options.add_argument(
        "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/141.0.0.0 Safari/537.36"
    )

    # ✅ 시스템 언어 (정상적인 로케일 표현)
    options.add_argument("--lang=ko-KR")

    # ✅ 자동화 플래그 제거
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option("useAutomationExtension", False)

    # ✅ WebDriver 감지 우회용 스크립트 (선택적)
    prefs = {"credentials_enable_service": False, "profile.password_manager_enabled": False}
    options.add_experimental_option("prefs", prefs)

    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=options
    )
    # 웹브라우저에서 url 열기
    driver.get(f"https://play.google.com/store/apps/details?id={app[1]}")

    # 요소가 실행 가능해질 때 까지 기다리기
    wait = WebDriverWait(driver, 10)

    # 평점 및 리뷰 옆의 -> 버튼이 활성화 되도록 스크롤을 1400px 아래로 내림
    driver.execute_script(f"window.scrollTo({0}, {1400})")
    time.sleep(2)

    # 평점 및 리뷰 옆의 -> 버튼 클릭
    button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[aria-label*="평점 및 리뷰 자세히 알아보기"]')))
    button.click()

    # 최신순으로 리뷰를 정렬하기 위해서 버튼 클릭
    wait.until(EC.element_to_be_clickable((By.ID, "sortBy_1"))).click()
    time.sleep(2)

    # 최신을 찾아 클릭
    wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'span[aria-label*="최신"]'))).click()
    time.sleep(2)

    # 스크롤을 내린 후 마지막 자료의 날짜를 찾아서 오늘 날짜부터 1주일 전 날짜와 비교하기

    # 리뷰가 담긴 창을 찾아서 Javascript로 10000px씩 아래로 스크롤
    today = datetime.today()
    end_date = today - timedelta(days=days)
    end_date = end_date.date()
    last_review_date = datetime.today().date()
    end = 1000
    len_list_old = len(driver.find_elements(By.CSS_SELECTOR, "div.RHo1pe"))
    while last_review_date > end_date:
        driver.execute_script(f"document.querySelector('.fysCi.Vk3ZVd').scrollTo(0, {end})")
        time.sleep(2)
        # 리뷰 목록 가져오기 div.RHo1pe
        review_list = driver.find_elements(By.CSS_SELECTOR, "div.RHo1pe")
        len_list_new = len(review_list)
        if len_list_old >= len_list_new:
            end += 10000
        else:
            len_list_old = len_list_new


        last_review_date = review_list[-1]
        last_review_date = last_review_date.find_element(By.CSS_SELECTOR, ".bp9Aid").get_attribute("innerHTML")
        last_review_date = to_date(last_review_date)

        print(len(review_list))
        print(last_review_date, end_date)


    print(len(review_list))
    print(end)

    # 리뷰 목록 가져오기 div.RHo1pe
    # review_list = driver.find_elements(By.CSS_SELECTOR, "div.RHo1pe")
    result = {}
    cols = ('리뷰일', '앱이름','별점', '사용자 리뷰','회사 응답')
    for review in review_list:

        # 리뷰일
        review_date = review.find_element(By.CSS_SELECTOR, ".bp9Aid").get_attribute("innerHTML")
        # 추출한 리뷰일을 날짜형 데이터로 변경
        review_date = to_date(review_date)

        # 별점
        rating = float(review.find_element(By.CSS_SELECTOR, 'div[aria-label*="별표 5개 만점에"]').get_attribute("aria-label").split()[3].replace("개를", ""))

        # 사용자 리뷰
        user_review = review.find_element(By.CSS_SELECTOR, ".h3YV2d").get_attribute("innerHTML")

        # 회사 응답
        try:
            company_reply = review.find_element(By.CSS_SELECTOR, ".ras4vb > div").get_attribute("innerHTML")
        except:
            company_reply = "회사 응답 없음"

        values = (review_date, app[0], rating, user_review, company_reply)
        for key, value in zip(cols, values):
            result.setdefault(key, []).append(value)
    driver.close()    
    df = pd.DataFrame(result)
    return df


In [23]:
for app in apps.items():
    result = app_review_extractor(app)
    display(result)

20
2026-02-09 2026-02-04
40
2026-02-08 2026-02-04
40
2026-02-08 2026-02-04
60
2026-02-06 2026-02-04
80
2026-02-05 2026-02-04
80
2026-02-05 2026-02-04
100
2026-02-03 2026-02-04
100
31000


,리뷰일,앱이름,별점,사용자 리뷰,회사 응답
0,2026-02-10,토스,1.0,아니 이거 업뎃 되긴해요??? 계속 업뎃 불가라고만 뜨고 안되는데????,"안녕하세요. 조수연 님, 토스팀입니다.기기에 캐시 및 데이터가 많아 앱 설치/업데이..."
1,2026-02-10,토스,1.0,토스뱅크 이중이체 토스뱅크내 통장 자동이체관리에서 등록한거와 앱 메뉴에서 자동이체 ...,"안녕하세요, 고객님. 토스뱅크입니다. 토스뱅크 서비스 이용 중 불편함을 겪으셨다면 ..."
2,2026-02-10,토스,4.0,"1. 포인트 주는 광고에 신경쓰길. 대행사에 던지는 줄 모르지만, 3초 보라면서 5...","안녕하세요. Jerry Eom 님, 토스팀입니다. 만족스러운 서비스를 제공하기 위해..."
3,2026-02-09,토스,4.0,은행 앱으로서는 잘사용 하고있습니다. 이런저런 해택도있고해서 좋은데요. 고양이 키우...,"안녕하세요. 몰라 님, 토스팀입니다. 만족스러운 서비스를 제공하기 위해 노력하였으나..."
4,2026-02-09,토스,1.0,쓰레기앱에 바퀴벌레보다 못한것들이 운영하는시스템 멀쩡하게 잘되는앱을 어떻게든 앱태크...,"안녕하세요. 클라나드 님, 토스팀입니다.먼저 토스를 이용해주셔서 감사합니다. 토스를..."
...,...,...,...,...,...
95,2026-02-03,토스,5.0,토스 라방 알림받기 표시 다시 있게 해주세여 들어가서 할려니 번거롭네요,안녕하세요. 토스팀입니다. 토스를 이용하시면서 느끼신 소중한 의견을 주셔서 감사합니...
96,2026-02-03,토스,5.0,갑자기 아이콘이 사라졌어요..play스토어 에 가서 다시 다운받으려고 하니까 다운등...,"안녕하세요. 박인선 님, 토스팀입니다. 아이콘이 사라져 많이 불편하셨을 것 같습니다..."
97,2026-02-03,토스,1.0,쓰레기토스절대하지마세요 5분동안전화먹통렉이심함 방금회원가입했는데갑자기폰이먹통 짜증!,안녕하세요. 토스팀입니다. 우선 원활한 이용에 불편을 드려 죄송합니다. 토스 앱 사...
98,2026-02-03,토스,5.0,혹시나 개선해주실수있는지 하는 마음으로 리뷰답니다. 제가 지금 핸드폰이 압수되서 컴...,안녕하세요. 윤지호님 토스팀입니다. 토스를 이용하시면서 느끼신 소중한 의견을 주셔서...


20
2026-02-07 2026-02-04
40
2026-02-04 2026-02-04
40
11000


,리뷰일,앱이름,별점,사용자 리뷰,회사 응답
0,2026-02-10,KB스타뱅킹,5.0,빠르게 이용핤 있어 좋아요,"안녕하세요, 고객님. KB스타뱅킹을 꾸준히 이용해주시고 소중한 시간을 내어 칭찬 글..."
1,2026-02-10,KB스타뱅킹,4.0,아주 편리하게 사용하고 있습니다,"고객님, 안녕하세요! KB스타뱅킹을 이용해 주셔서 진심으로 감사합니다. 저희 KB스..."
2,2026-02-09,KB스타뱅킹,5.0,빠름잘되었음,"안녕하세요, 고객님. KB스타뱅킹을 꾸준히 이용해주시는 고객님께 진심으로 감사의 말..."
3,2026-02-09,KB스타뱅킹,5.0,깔끔한 진행 굿굿,"안녕하세요, 고객님. KB스타뱅킹을 이용해주시고, 바쁘신 시간 중에도 소중한 칭찬글..."
4,2026-02-09,KB스타뱅킹,1.0,기술력 진짜 너무 떨어진다. 불을 끄고 옮기고 키고 옮기고 수십번을 해도 실물신분증...,"안녕하세요, 신분증 촬영에 불편을 드려 죄송합니다. 신분증 촬영에 지속적으로 어려움..."
5,2026-02-09,KB스타뱅킹,1.0,접속이 안되고 로딩창만 계속 떠있어요 ㅡㅡ 그래서 앱 눌러 목록 띄워서 조회선택해야...,"안녕하세요 고객님, KB스타뱅킹 이용에 불편을 드려 죄송합니다. 고객님께서 남겨주신..."
6,2026-02-09,KB스타뱅킹,5.0,보안이 철저하고 설명해주는 것도 간편해서 좋아요,"안녕하세요, 고객님. KB스타뱅킹을 이용해주시고, 바쁘신 시간 중에도 소중한 칭찬글..."
7,2026-02-09,KB스타뱅킹,1.0,최악의 앱,"고객님, 안녕하세요! KB스타뱅킹을 이용해주셔서 진심으로 감사합니다. 앞으로도 고객..."
8,2026-02-09,KB스타뱅킹,5.0,좋아요,"안녕하세요, 고객님. KB스타뱅킹을 꾸준히 이용해주시고 소중한 시간을 내어 칭찬 글..."
9,2026-02-09,KB스타뱅킹,5.0,최고,"안녕하세요, 고객님. KB스타뱅킹을 이용해주시고, 바쁘신 시간 중에도 소중한 칭찬글..."


20
2026-01-29 2026-02-04
20
11000


,리뷰일,앱이름,별점,사용자 리뷰,회사 응답
0,2026-02-10,하나원큐,4.0,심플이체 진짜 편한데 이거 왜 서비스 중단했을까요? 자주쓰는계좌 서비스는 금액이나 ...,회사 응답 없음
1,2026-02-10,하나원큐,1.0,왜케불편함?,회사 응답 없음
2,2026-02-08,하나원큐,1.0,타은행어플의 280분에1수준의 로딩속도,회사 응답 없음
3,2026-02-07,하나원큐,5.0,편하고 빠룸,회사 응답 없음
4,2026-02-06,하나원큐,5.0,굿,회사 응답 없음
5,2026-02-06,하나원큐,5.0,사용이.편해요.,회사 응답 없음
6,2026-02-04,하나원큐,1.0,계좌없으면 하나꺼 만들꺼냐 다른계좌로 인증 할꺼냐 해서 계좌 많아서 타계좌로 인증하...,안녕하세요. 하나은행입니다.하나원큐 앱 서비스를 이용해 주셔서 감사드립니다. 우선 ...
7,2026-02-04,하나원큐,1.0,삼성에서 권장하는 자가수리후 앱의 접속을 하지못하고 있습니다 ㅡㅅㅡ 액정바꿨을 뿐인...,회사 응답 없음
8,2026-02-04,하나원큐,5.0,간편하고 신속해서 넘 좋아요,회사 응답 없음
9,2026-02-03,하나원큐,5.0,굿,회사 응답 없음


20
2026-02-05 2026-02-04
40
2026-01-25 2026-02-04
40
11000


,리뷰일,앱이름,별점,사용자 리뷰,회사 응답
0,2026-02-09,뱅크샐러드,5.0,어떨결에 뱅샐이 알바시키기 시작했는데 돈은 얼마 못벌지만ㅋㅋㅋ 불꺼진 곳에서 눈 깜...,"안녕하세요? 뱅크샐러드 고객감동팀입니다. 뱅크샐러드를 이용해 주시고, 리뷰 남겨 주..."
1,2026-02-09,뱅크샐러드,2.0,"수년간 잘 써오고 있는데, 이번 업데이트는 정말 불편합니다. 수기입력 시 시간 자동...",회사 응답 없음
2,2026-02-09,뱅크샐러드,2.0,이번 업데이트로 인터페이스가 더 불편해졌어요..,회사 응답 없음
3,2026-02-08,뱅크샐러드,5.0,업뎃 된거 너무좋아요.. 진짜 1년동안 기다렸어요ㅠ 예전에 업뎃됬다가 다시 이전버전...,회사 응답 없음
4,2026-02-08,뱅크샐러드,1.0,UI 이전으로 되돌려주세요 스크롤로 선택하는 게 너무 불편합니다.,회사 응답 없음
5,2026-02-08,뱅크샐러드,3.0,오늘 아침부터 입출금 내역 실시간반영이 안되는데 확인해주세요,회사 응답 없음
6,2026-02-07,뱅크샐러드,4.0,네이버쇼핑에서 네이버 페이 포인트로 결제를 하면 네이버페이 간편결제(포인트)라고 표...,회사 응답 없음
7,2026-02-07,뱅크샐러드,5.0,최근 업데이트 된 후 가계부 목록에 리스트 내 폰트가 뭔가 좀 더 light해진듯 ...,회사 응답 없음
8,2026-02-07,뱅크샐러드,1.0,ui를 왜 굳이 바꾸는 건지.....,회사 응답 없음
9,2026-02-07,뱅크샐러드,4.0,카카오페이의 경우 지역화폐 지출(코나아이) 내역도 다 연동이 되는데 뱅크샐러드의 경...,회사 응답 없음


20
2026-01-16 2026-02-04
20
11000


,리뷰일,앱이름,별점,사용자 리뷰,회사 응답
0,2026-02-09,핀다,5.0,감사합니다.급전이 필요했는데. 낮은 금리로 당일입금 시켜주어 대단히 감사합니다. 신...,"안녕하세요, 김진철님. 핀다 고객경험팀입니다 :)\n급하게 필요하신 자금을 낮은 금..."
1,2026-02-05,핀다,5.0,여러가지 대출상품 비교도 가능하고 간편하고 빠르게 대출에서 입금까지 너무너무 편하고...,"안녕하세요, 버리스타님. 핀다AI 입니다 :)\n여러 대출상품을 간편하게 비교하고 ..."
2,2026-01-29,핀다,5.0,정확하고 빠릅니다 간편하구요,"안녕하세요. 이놈자네가 주임원산가님, 핀다 고객경험팀입니다 :)\n정확하고 빠른 서..."
3,2026-01-29,핀다,1.0,과대 광고 쓰레기 앱,"안녕하세요, 양상일님. 핀다 고객경험팀입니다. 기대에 미치지 못한 경험을 하셨다니 ..."
4,2026-01-29,핀다,5.0,토스에서 안되었는데..너무감사합니다,"안녕하세요, 전미희님. 핀다AI 입니다 :)\n타 서비스에서 어려움을 겪으셨는데 핀..."
5,2026-01-27,핀다,5.0,최고입니다,"안녕하세요, 김지훈님. 핀다AI 입니다 :)\n""최고입니다""라는 따뜻한 칭찬 정말 ..."
6,2026-01-27,핀다,5.0,다른데서 다 부결났는데 핀다에서 겨우 급한돈 대출받앗네요 ㅜㅜ,"안녕하세요, 김동현님. 핀다AI 입니다 :)\n빠르게 답변을 드리고자 핀다AI가 먼..."
7,2026-01-27,핀다,4.0,감사합니다,"안녕하세요, 윤진원님. 핀다 고객경험팀입니다 :) 핀다를 이용해 주시고 소중한 리뷰..."
8,2026-01-27,핀다,5.0,다른 어플보다 확실히 다양한 루트를 안내해주니 승인률이 높음,"안녕하세요, 정현호님. 핀다AI 입니다 :)\n빠르게 답변을 드리고자 핀다AI가 먼..."
9,2026-01-26,핀다,5.0,빠르고 간단하게 결정해주는 광주은행 비대면 감사합니다. 26.1.26,"안녕하세요, WH님. 핀다AI 입니다 :)\n빠르게 답변을 드리고자 핀다AI가 먼저..."
